In [1]:
# Import library
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import smtplib

In [2]:
# Set up Selenium
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(service=service, options=options)

# Connect to a website
url = 'https://www.amazon.com/Gloria-Vanderbilt-Amanda-Average-Madison/dp/B079GTKB7B/'
driver.get(url)

# Parse page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

# Extract title and price
title = soup.find('span', {"id": "productTitle"})
title = title.get_text(strip=True) if title else "No Title"

price = soup.select_one("span.a-offscreen")
price = price.get_text(strip=True)[1:] if price else "No Price"


In [3]:
# pull the data to CSV
import datetime
date = datetime.date.today()

import csv

header = ['Title','Price','Date']
data = [title,price,date]

In [18]:
# write the data to the csv file named amazonDataset

with open('amazonDataset.csv','w',newline='',encoding='UTF8') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  writer.writerow(data)

In [9]:
# create a function to append a data with scheduler
def updatePriceData():

  url = 'https://www.amazon.com/Gloria-Vanderbilt-Amanda-Average-Madison/dp/B079GTKB7B/'
  driver.get(url)

  soup = BeautifulSoup(driver.page_source, "html.parser")

  # Extract title and price
  title = soup.find("span", {"id": "productTitle"})
  title = title.get_text(strip=True) if title else "No Title"


  price = soup.select_one("span.a-offscreen")
  price = price.get_text(strip=True)[1:] if price else "No Price"

  # Add a data extraction data on the third column
  import datetime
  date = datetime.date.today()

  # Append the data extracted on the existing csv file while the function is called

  with open('amazonDataset.csv','a+',newline='',encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)


In [11]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_mail(to_email, price):
    sender_email = os.getenv("EMAIL_ADDRESS")
    sender_password = os.getenv("EMAIL_PASSWORD")
    if not sender_password:
        print("Error: EMAIL_PASSWORD is not set!")
        return
    
    subject = "Price Alert: Price Dropped Below $15!"
    body = f"The price has dropped to ${price}. Check it out now!"
    
    # Setup email content
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = to_email
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))
    
    try:
        # Use TLS with port 587
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()  # Upgrade to secure connection
        server.login(sender_email, sender_password)
        server.send_message(msg)
        print("Email sent successfully!")
    except smtplib.SMTPAuthenticationError:
        print("Error: Authentication failed. Check your email and password.")
    except smtplib.SMTPException as e:
        print(f"SMTP error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        try:
            server.quit()
        except:
            pass

In [20]:
import time

last_price = None

while True:
    try:
        updatePriceData()
        
        if price is not None and float(price) < 20 and price != last_price:
            print(f"Price dropped to ${price}, sending email...")
            send_mail("ashtipay@gmail.com", price)
            last_price = price
        else:
            print(f"Current price is ${price}. No email sent.")
        
    except Exception as e:
        print(f"Error while updating price: {e}")
    
    time.sleep(60)  # Check every 60 seconds

Price dropped to $19.22, sending email...
Email sent successfully!
Current price is $19.22. No email sent.


KeyboardInterrupt: 